# multi

In nbs/API/multi.ipynb

This module provides functionality for visualizing multiple DABEST contrast objects simultaneously using advanced visualization techniques like vortexmaps and forest plots.
- order: 11

In [ ]:
#| default_exp multi

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev
nbdev.nbdev_export()

In [ ]:
#| export
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

## MultiContrast Class

The `MultiContrast` class enables visualization of multiple contrast objects in grid-based layouts.



In [ ]:
#| export
class MultiContrast:
    """
    Multiple contrast objects enabling multi-contrast visualizations.
    
    This class takes in arrays of dabest contrast objects (e.g., mean_diff, delta_delta, mini_meta)
    and creates grid-based visualizations like forest plots and vortexmaps.
    """
    
    def __init__(self, contrasts, labels=None):
        """
        Initialize MultiContrast object.
        
        Parameters
        ----------
        contrasts : list
            List of dabest contrast objects (e.g., from .mean_diff, .delta_delta, etc.)
        labels : list, optional
            Labels for each contrast. If None, defaults will be generated.
        """
        self.contrasts = self._check_contrasts(contrasts)
        self.labels = labels or self._generate_default_labels()
        self._effect_sizes_cache = None
        
    def _check_contrasts(self, contrasts):
        """Check that all objects are valid dabest contrast objects."""
        if not isinstance(contrasts, (list, tuple)):
            raise TypeError("contrasts must be a list or tuple")
            
        if len(contrasts) == 0:
            raise ValueError("contrasts cannot be empty")
            
        return list(contrasts)
        
    def _generate_default_labels(self):
        """Generate default labels like 'Contrast 1', 'Contrast 2', etc."""
        return [f"Contrast {i+1}" for i in range(len(self.contrasts))]
        
    def __repr__(self):
        return f"MultiContrast with {len(self.contrasts)} contrasts: {self.labels}"

## Loading Function

In [ ]:
#| export
def combine(contrasts, labels=None):
    """
    Load multiple contrast objects for multi-contrast visualization.
    
    Parameters
    ----------
    contrasts : list
        List of dabest contrast objects (e.g., from .mean_diff, .delta_delta, etc.)
    labels : list, optional
        Labels for each contrast. If None, defaults will be generated.
        
    Returns
    -------
    MultiContrast
        Object containing the contrasts and methods for visualization
    """
    return MultiContrast(contrasts, labels)

In [ ]:
#| export
def _parse_contrast_structure(contrasts, labels=None):
    """
    Parse contrast structure and normalize to 2D format for unified handling.
    
    Returns
    -------
    dict with keys:
        - 'contrasts_2d': always 2D structure [[c1, c2], [c3, c4]] or [[c1, c2, c3]]
        - 'n_rows': number of rows
        - 'n_cols': number of columns  
        - 'row_labels': labels for rows
        - 'col_labels': labels for columns
        - 'was_1d': bool indicating if input was originally flat
    """
    if isinstance(contrasts[0], (list, tuple)):
        # Already 2D - keep as is
        contrasts_2d = contrasts
        n_rows = len(contrasts)
        n_cols = len(contrasts[0])
        was_1d = False
        
        # Handle 2D labels
        if labels and isinstance(labels[0], (list, tuple)):
            row_labels = [labels[i][0] for i in range(n_rows)]
            col_labels = labels[0][1:]
        else:
            row_labels = [f"Row {i+1}" for i in range(n_rows)]
            col_labels = [f"Col {j+1}" for j in range(n_cols)]
            
    else:
        # 1D - force into single row 2D structure
        contrasts_2d = [contrasts]  # Wrap in single row
        n_rows = 1
        n_cols = len(contrasts)
        was_1d = True
        
        # Handle 1D labels
        flat_labels = labels or [f"Contrast {i+1}" for i in range(n_cols)]
        row_labels = [" "]  # Empty row label for single row
        col_labels = flat_labels
        
    return {
        'contrasts_2d': contrasts_2d,
        'n_rows': n_rows,
        'n_cols': n_cols,
        'row_labels': row_labels,
        'col_labels': col_labels,
        'was_1d': was_1d
    }

## Vortexmap Visualization

The vortexmap creates spiral heatmaps showing the distribution of bootstrap samples for each contrast.

In [ ]:
#| export
def _sample_bootstrap(bootstrap, m, n, reverse_neg, abs_rank, chop_tail):
    """Sample bootstrap values and prepare for spiral visualization."""
    bootstrap_sorted = sorted(bootstrap)
    chop_tail_int = int(np.ceil(len(bootstrap_sorted) * chop_tail / 100))
    bootstrap_sorted = bootstrap_sorted[chop_tail_int : len(bootstrap_sorted) - chop_tail_int]
    
    ranks_to_look = np.linspace(0, len(bootstrap_sorted), m * n, dtype=int)   
    ranks_to_look[0] = 1
    
    if np.sum(np.array(bootstrap_sorted) > 0) < len(bootstrap_sorted) / 2:
        if reverse_neg:
            bootstrap_sorted = bootstrap_sorted[::-1]
    
    if abs_rank:
        bootstrap_sorted = sorted(bootstrap_sorted, key=abs)
    
    long_ranks = [bootstrap_sorted[r - 1] for r in ranks_to_look]
    return long_ranks

In [ ]:
#| export
def _spiralize(fill, m, n):
    """Convert linear array into spiral pattern."""
    i = 0
    j = 0
    k = 0
    array = np.zeros((m, n))
    
    while m > 0 and k < len(fill):
        jj = j
        ii = i
        
        # Right
        for j in range(j, n):
            if k >= len(fill):
                break
            array[i, j] = fill[k]
            k += 1
        
        # Down
        for i in range(ii + 1, m):
            if k >= len(fill):
                break
            array[i, j] = fill[k]
            k += 1
        
        # Left
        for j in range(n - 2, jj - 1, -1):
            if k >= len(fill):
                break
            array[i, j] = fill[k]
            k += 1
        
        # Up
        for i in range(m - 2, ii, -1):
            if k >= len(fill):
                break
            array[i, j] = fill[k]
            k += 1
        
        m -= 1
        n -= 1
        j += 1
        
    return array

In [ ]:
#| export
def vortexmap(multi_contrast, n=21, sort_by=None, cmap = 'vlag', vmax = None, vmin = None, 
              reverse_neg=True, abs_rank=False, chop_tail=0, ax=None,fig_size=None, **kwargs):
    """
    Create a vortexmap visualization of multiple contrasts.
    
    Parameters
    ----------
    multi_contrast : MultiContrast
        Object containing multiple contrast objects
    n : int, default 21
        Size of each spiral (n x n grid per contrast)
    sort_by : list, optional
        Order to sort contrasts by
    vmax, vmin : float, default None, None
        Color scale limits
    reverse_neg : bool, default True
        Whether to reverse negative values
    abs_rank : bool, default False
        Whether to rank by absolute value
    chop_tail : float, default 0
        Percentage of extreme values to exclude
    ax : matplotlib.Axes, optional
        Existing axes to plot on
        
    Returns
    -------
    tuple
        (figure, axes, mean_delta_dataframe) if ax is None, 
        else (axes, mean_delta_dataframe)
    """
    structure = _parse_contrast_structure(multi_contrast.contrasts, multi_contrast.labels)
    
    n_rows = structure['n_rows']
    n_cols = structure['n_cols']
    col_labels = structure['col_labels']
    row_labels = structure['row_labels']
    contrasts_2d = structure['contrasts_2d']

    spirals = pd.DataFrame(np.zeros((n_rows * n, n_cols * n)))
    
    mean_delta = pd.DataFrame(np.zeros((n_rows, n_cols)), 
                             columns=col_labels, 
                             index=row_labels)
    
    for i in range(n_rows):
        for j in range(n_cols):
            contrast_idx = sort_by[j] if sort_by is not None else j
            contrast = contrasts_2d[i][contrast_idx]
            
            # Get bootstrap samples based on contrast type
            if hasattr(contrast, 'delta2') and contrast.delta2:
                bootstrap = contrast.delta_delta.bootstraps_delta_delta
            else:
                bootstrap = contrast.results.bootstraps[0]
            
            long_ranks = _sample_bootstrap(bootstrap, n, n, reverse_neg, abs_rank, chop_tail)
            spiral = _spiralize(long_ranks, n, n)
            spirals.iloc[i*n:i*n+n, j*n:j*n+n] = spiral
            mean_delta.iloc[i, j] = np.mean(long_ranks)
    
    if ax is None:
        f, a = plt.subplots(1, 1)
    else:
        a = ax
    if vmax is None:
        vmax = np.max(spirals.values)
    if vmin is None:
        vmin = np.min(spirals.values)
    if structure['was_1d']:
        cbar_orientation = 'horizontal'
        cbar_location = 'top'
    else:
        cbar_orientation = 'vertical'
        cbar_location = 'right'
    
    # Create heatmap
    sns.heatmap(spirals, cmap=cmap, cbar_kws={"shrink": 1, "pad": .17, "orientation": cbar_orientation, "location": cbar_location}, 
                ax=a, center = 0, vmax=vmax, vmin=vmin,  **kwargs)
    
    # Set labels
    a.set_xticks(np.linspace(n/2, n_cols*n-n/2, n_cols))
    a.set_xticklabels(col_labels, rotation=45, ha='right')

    if structure['was_1d']:
        a.set_xlabel('Contrasts')
        a.set_ylabel(' ')
        a.set_yticks([])
        a.set_yticklabels([])
    else:
        a.set_yticks(np.linspace(n/2, n_rows*n-n/2, n_rows))
        a.set_yticklabels(row_labels, ha='right', rotation=0)

    if ax is None:
        f.gca().set_aspect('equal')
        if fig_size is None:
            f.set_size_inches(n_cols/3, n_rows/3)
        else:
            f.set_size_inches(fig_size)
        return f, a, mean_delta
    else:
        return a, mean_delta
    


In [ ]:
#| export
__all__ = ['MultiContrast', 'combine', 'vortexmap']
